# Feed-forward neural network v3
- a.k.a. Multi_Layer Perceptrons(MLP), Fully-connected network
- [variable_scope description (korean)](https://tensorflowkorea.gitbooks.io/tensorflow-kr/content/g3doc/how_tos/variable_scope/)

In [21]:
import os
import time
import random
import numpy as np
# import tensorflow as tf
import tensorflow.contrib.imperative as tf

from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.python import debug as tf_debug
from tensorflow.python.debug.lib import debug_utils

In [22]:
# conf
dump_local = True

In [26]:
# debug
sess = tf.Session()

if dump_local is True:
    dump_root_dir = os.path.abspath("./dump/tfdbg_dumps/%s" % time.time())
    os.makedirs(dump_root_dir)
    sess = tf_debug.LocalCLIDebugWrapperSession(sess, dump_root=dump_root_dir)
    sess.add_tensor_filter("has_inf_or_nan", tf_debug.has_inf_or_nan)

In [27]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
# Check out https://www.tensorflow.org/get_started/mnist/beginners for
# more information about the mnist dataset

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [28]:
X_train = mnist.train.images
Y_train = mnist.train.labels
X_valid = mnist.validation.images
Y_valid = mnist.validation.labels
X_test = mnist.test.images
Y_test = mnist.test.labels

In [29]:
dim_X = X_train.shape[1]
pixel_X = int(np.sqrt(dim_X)) # np.sqrt의 출력이 float32이므로, 이를 int 자료형으로 변경
dim_Y = Y_train.shape[1]
print(dim_X, pixel_X, dim_Y)

784 28 10


In [30]:
# Configuration
learning_rate = 0.001
training_epochs = 15
batch_size = 100
hidden_dim_1 = 300
hidden_dim_2 = 100

In [31]:
X = tf.placeholder(tf.float32, [None, dim_X], name="Inputs")
Y = tf.placeholder(tf.float32, [None], name="Outputs")

In [32]:
# weights & bias for nn layers
W1 = tf.Variable(np.random.normal(
    size=[dim_X, hidden_dim_1]), dtype=tf.float32, name="weight_1")
b1 = tf.Variable(np.random.normal(
    [hidden_dim_1]), dtype=tf.float32, name="bias_1")
h1 = tf.nn.relu(tf.matmul(X, W1) + b1)

In [33]:
# weights & bias for nn layers
W2 = tf.Variable(np.random.normal(
    size=[hidden_dim_1, hidden_dim_2]), dtype=tf.float32, name="weight_2")
b2 = tf.Variable(np.random.normal(
    [hidden_dim_2]), dtype=tf.float32, name="bias_2")
h2 = tf.nn.relu(tf.matmul(h1, W2) + b2)

In [34]:
# weights & bias for nn layers
W_out = tf.Variable(np.random.normal(
    size=[hidden_dim_2, dim_Y]), dtype=tf.float32, name="weight_3")
b_out = tf.Variable(np.random.normal(
    [dim_Y]), dtype=tf.float32, name="bias_3")
logits =tf.matmul(h2, W_out) + b_out

In [35]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

UnimplementedError: ApplyGradientDescent op not supported in imperative graph

In [ ]:
sess.run(tf.global_variables_initializer())

In [ ]:
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples / batch_size)
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        _cost, _ = sess.run([cost, optimizer], feed_dict={X:batch_xs, Y:batch_ys})
        avg_cost += _cost / total_batch
    
    print("Epoch: {} \t cost: {:0.4f}".format(epoch, avg_cost))

In [ ]:
sess.close()